In [1]:
import argparse
import collections
import torch
import numpy as np
import data_loader.data_loaders as module_data
import data_loader.preprocess as module_preprocess
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
from trainer import Trainer


# fix random seeds for reproducibility
SEED = 123
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

In [2]:
import json
config = ConfigParser(json.load(open('config.json', 'r')))

In [3]:
logger = config.get_logger('train')
# preprocess
# config.init_obj('preprocess', module_preprocess)

# setup data_loader instances
data_loader = config.init_obj('data_loader', module_data, save_dir=config.save_dir)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release 

In [4]:
data_loader.TEXT.vocab.itos[:5]

['<UNK>', '<PAD>', '<START>', '<END>', '.']

In [5]:
print(vars(data_loader.dataset.examples[0]))

{'talk': ['can', 'we', 'make', 'this', 'quick', '?', ' ', 'roxanne', 'korrine', 'and', 'andrew', 'barrett', 'are', 'having', 'an', 'incredibly', 'horrendous', 'public', 'break-', 'up', 'on', 'the', 'quad', '.', ' ', 'again', '.'], 'response': ['well', ',', 'i', 'thought', 'we', "'d", 'start', 'with', 'pronunciation', ',', 'if', 'that', "'s", 'okay', 'with', 'you', '.']}


In [6]:
print(vars(data_loader.TEXT))

{'sequential': True, 'use_vocab': True, 'init_token': '<START>', 'eos_token': '<END>', 'unk_token': '<UNK>', 'fix_length': 32, 'dtype': torch.int64, 'preprocessing': None, 'postprocessing': None, 'lower': True, 'tokenizer_args': (<bound method ChatbotDataLoader._tokenizer of <data_loader.data_loaders.ChatbotDataLoader object at 0x10ebea190>>, 'en'), 'tokenize': <bound method ChatbotDataLoader._tokenizer of <data_loader.data_loaders.ChatbotDataLoader object at 0x10ebea190>>, 'include_lengths': False, 'batch_first': False, 'pad_token': '<PAD>', 'pad_first': False, 'truncate_first': False, 'stop_words': None, 'is_target': False, 'vocab': <torchtext.vocab.Vocab object at 0x132e29fa0>}


In [41]:
print(config.__dict__)

{'_config': {'name': 'Chatbot', 'n_gpu': 1, 'arch': {'type': 'ChatbotModel', 'args': {}}, 'preprocess': {'type': 'ChatbotDataPreprocess', 'args': {'data_dir': 'data/cornell movie-dialogs corpus'}}, 'data_loader': {'type': 'ChatbotDataLoader', 'args': {'data_dir': 'data/cornell movie-dialogs corpus', 'filename': 'formatted_movie_lines.csv', 'batch_size': 128, 'sent_len': 256, 'init_token': '<START>', 'eos_token': '<END>', 'min_freq': 5, 'shuffle': True, 'validation_split': 0.1, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.001, 'weight_decay': 0, 'amsgrad': True}}, 'loss': 'nll_loss', 'metrics': ['accuracy', 'top_k_acc'], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamma': 0.1}}, 'trainer': {'epochs': 10, 'save_dir': 'saved/', 'save_period': 1, 'verbosity': 2, 'monitor': 'min val_loss', 'early_stop': 2, 'tensorboard': True}}, 'resume': None, '_save_dir': PosixPath('saved/models/Chatbot/1013_210419'), '_log_dir': PosixPath('saved/log/Chatbot/1013_21

In [7]:
data_loader.train_iter.

<bound method Iterator.data of <torchtext.data.iterator.BucketIterator object at 0x132e29f70>>

In [48]:
with open("TEXT.Field","wb")as f:
     dill.dump(data_loader.TEXT, f)
# torch.save(data_loader.TEXT, 'text.dill', pickle_module=dill)

TypeError: 'generator' object is not callable

In [49]:
print(data_loader.TEXT)

In [54]:
from torchtext.data import Field

In [55]:
question_field = Field(sequential=True)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [56]:
torch.save(question_field, 'question.Field')

In [57]:
import pickle

In [58]:
pickle.dump(data_loader.TEXT.vocab, open('test.vocab', 'wb'))

In [59]:
torch.save(data_loader.TEXT.vocab, 'test.vocab')

NameError: name 'sent' is not defined

In [17]:
l = [(0,1), (2,3)]
for i, j in zip(*l):
    print(i, j)

0 2
1 3


In [22]:
datasets = data_loader.dataset.split(split_ratio=0.9)

AttributeError: 'BucketIterator' object has no attribute 'split'

In [27]:
from torchtext.data import TabularDataset, Field
TEXT = Field(
    sequential=True,
    lower=True,
    include_lengths=True
)

dataset = TabularDataset(
    path='data/cornell movie-dialogs corpus/formatted_movie_lines.csv',
    format='csv',
    fields={
        'talk': ('talk', TEXT),
        'response': ('response', TEXT)
    },
    csv_reader_params={'delimiter': '\t'},
    skip_header=False
)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release 

In [45]:
splits = dataset.split(split_ratio=0.9)

ValueError: Split ratio must be float or a list, got <class 'int'>

In [32]:
len(splits[0].examples)

199154

In [33]:
len(splits[1].examples)

22128

In [34]:
len(dataset.examples)

221282

In [42]:
data_loader.TEXT.vocab.stoi['<PAS>']

0

In [50]:
from torchtext.data import BucketIterator

In [51]:
train_iter = BucketIterator(splits[0], 2)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [57]:
valid_iter = BucketIterator(splits[1], 8)

In [59]:
for i in valid_iter:
    print(i)

/Users/hlu/anaconda3/envs/torch/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


AttributeError: 'Field' object has no attribute 'vocab'